In [ ]:
from datascience import *
import numpy as np
import matplotlib

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter("ignore")

## Review

### The $k$-Nearest Neighbors algorithm

#### Find the “distance” between the example and one training point

##### We use $\text{D}$, the **Euclidean distance:**

$$ \text{D} = \sqrt{(x_0-x_1)^2 + (y_0-y_1)^2} $$

In [ ]:
def distance(point1, point2):
    """Returns the distance between point1 and point2
    where each argument is an array 
    consisting of the coordinates of the point"""
    return np.sum((point1-point2)**2) ** 0.5

#### Find the “distance” between the example and each observation in the training set

In [ ]:
def all_distances(training, new_row):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.drop('Class')
    def distance_from_new(row):
        return distance(make_array(new_row), make_array(row))
    return attributes.apply(distance_from_new)

#### Augment the training data table with a column containing all the distances

In [ ]:
def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))

#### The smaller $\text{D}$ is, the nearer the neighbor! Find the $k$-nearest neighbors using table manipulations.

In [ ]:
def nearest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    nearest_neighbors_table = sorted_by_distance.take(np.arange(k))
    return nearest_neighbors_table

#### Take a majority vote of the $k$-nearest neighbors. Assign the new point to the majority class

In [ ]:
def majority(nearest_neighbors_table, class_name):
    return nearest_neighbors_table.group(class_name).sort('count', 
                                                         descending=True).column(class_name).item(0)

#### A function to run the complete algorithm!

In [ ]:
def one_knn(training, class_name, new_point, k):
    nearest_neighbors_table = nearest(training, new_point, k)
    return majority(nearest_neighbors_table, class_name)

____

### `ckd` dataset

In [ ]:
ckd = Table.read_table('ckd.csv').relabeled('Blood Glucose Random', 'Glucose')
ckd.sample(3).show(3)

In [ ]:
ckd = ckd.select('Hemoglobin','Glucose','White Blood Cell Count', 'Class')

_____

### `banknotes` dataset

In [ ]:
banknotes = Table().read_table('banknote.csv')
banknotes.sample(3).show()

In [ ]:
first_banknote = banknotes.drop('Class').row(0)
first_banknote

**Distance in 4 dimensions**

$$ \text{D} = \sqrt{(w_0-w_1)^2 + (x_0-x_1)^2 + (y_0-y_1)^2 + (z_0-z_1)^2} $$

In [ ]:
nearest(banknotes,first_banknote,1)

**Discussion Question**

## New material

In [ ]:
patients = Table.read_table('breast-cancer.csv').drop('ID')

In [ ]:
patients.sample(5).show()

In [ ]:
patients.num_columns

### Exploration

In [ ]:
patients.group('Class')

In [ ]:
patients.scatter('Bland Chromatin', 'Single Epithelial Cell Size', group='Class')
plt.title('You are witnessing something called overplotting!');

Do not worry about how to use the code below. Just understand that it makes a new table containing data that help out the previous visualization!

In [ ]:
def randomize_column(a):
    return a + np.random.normal(0.0, 0.09, size=len(a))

jittered = Table().with_columns([
        'Bland Chromatin (jittered)', 
        randomize_column(patients.column('Bland Chromatin')),
        'Single Epithelial Cell Size (jittered)', 
        randomize_column(patients.column('Single Epithelial Cell Size')),
        'Class',
        patients.column('Class')
    ])

In [ ]:
jittered.scatter(0, 1, group='Class')

____

### Prediction

In [ ]:
train,test = patients.split(k = round(patients.num_rows/2))

#### **Task**: Calculate the Euclidean distance between an example test point and every point in the training set.

In [ ]:
example = test.drop('Class').row(2)

**Discussion Question**

In [ ]:
table_with_distances(train, example)

##### **Task**: Find the 5-nearest neighbors!

In [ ]:
nearest(train, example, 5)

#### **Task**: Find the majority class!

In [ ]:
majority(nearest(train, example, 5), 'Class')

#### **Task**: Repeat this process for every point in the testing set!

In [ ]:
one_knn(train, 'Class', test.drop('Class').row(2),  5)

There are a few ways to do this. Today, we will use a for-loop method because it requires writing less functions.

In [ ]:
def knn(training, class_name, testing, k):
    """Returns the test predictions in the test set in a table, along with their actual classifications"""
    
    predictions = make_array()
    for i in np.arange(testing.num_rows):
        predictions = np.append(predictions, one_knn(training, 
                                                     class_name, 
                                                     testing.drop(class_name).row(i), 
                                                     k))

    knn_table = Table().with_columns('Actual', testing.column(class_name),
                                     'Predicted', predictions)
    
    return knn_table 

In [ ]:
results = knn(train, 'Class', test, 5)
results

#### **Task**: Evaluate the accuracy of the classifier with these results

$$\text{Misclassification Rate (MCR)} = \text{percentage of testing points that are misclassified}$$

Practice [3 min]: Find the mis-classification rate for the 5-nearest neighbors algorithm!

### Which $k$ should we choose?

We can figure out which one based on the evaluation metric, $\text{MCR}$. 

In [ ]:
def which_k(training, class_name, testing, k_values):

    mc_rates = make_array()

    for i in np.arange(np.size(k_values)):

        results = knn(training, class_name, testing, k_values.item(i))
        new_rate = ## input your misclsassificaton rate method here!
        mc_rates = np.append(mc_rates, new_rate)

    k_table = Table().with_columns("k", k_values,
                                   "MCR", mc_rates)
                             
    return k_table

In [ ]:
k_s = make_array(1,3,5,7,9,11,13,15)

In [ ]:
k_table = which_k(train, 'Class', test, k_s)

In [ ]:
k_table

**Discussion Question**: What plot type would work best to visualize how the $\text{MCR}$ increases and decreases as the number of neighbors increases?

In [ ]:
k_table.plot('k','MCR')
plt.title('The best k for the job is...?');

#### **Task**: See where our classifier went wrong.

**Discussion Question**: How many rows and columns will the table produced by the following command have?

```
results.group(['Actual', 'Predicted'])
```


In [ ]:
results.group(['Actual', 'Predicted'])

### If you are purely interested in prediction... standardize!

In [ ]:
def standard_units(x):
    return (x - np.average(x)) / np.std(x)

In [ ]:
ckd.sample(3).show(3)

In [ ]:
ckd_train,ckd_test = ckd.split(k = round(ckd.num_rows/2))

In [ ]:
which_k(ckd_train, 'Class', ckd_test, k_s)

In [ ]:
ckd_standardized = ckd.select('Class').with_columns(
    'Glucose_su', standard_units(ckd.column('Glucose')),
    'Hemoglobin_su', standard_units(ckd.column('Hemoglobin')),
    'WBC_su', standard_units(ckd.column('White Blood Cell Count'))
)

In [ ]:
ckd_standardized

In [ ]:
ckd_train,ckd_test = ckd_standardized.split(k = round(ckd.num_rows/2))

In [ ]:
which_k(ckd_train, 'Class', ckd_test, k_s)

**Discussion Question**